# Data Cleaning Listado de Productores Autorizados

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1) for file in csv_files]

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    print(f"Previous text: {text}")
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    print(f"Cleaned text: {text}")
    return text

# 1. Lectura de los datos

Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252')

Lectura del listado de productores autorizados

In [4]:
listado_productores = load_datasets('../../data/productores_autorizados/')

# 2. Limpieza de los datos

## 2.1 INEGI

In [5]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [6]:
# Revisamos las primeras filas del dataset
dataset_inegi.head()

,MAPA,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS
0,10010001,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.871874,-102.373530,1901,F13D18,5,*,*,2
2,10010096,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,96,Agua Azul,R,"21°53´01.522N""","102°21´25.639W""",21.883756,-102.357122,1861,F13D18,41,24,17,12
3,10010100,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,100,Rancho Alegre,R,"21°51´16.556N""","102°22´21.884W""",21.854599,-102.372745,1879,F13D18,0,0,0,0
4,10010102,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,102,Los Arbolitos [Rancho],R,"21°46´48.650N""","102°21´26.261W""",21.780180,-102.357295,1861,F13D18,8,*,*,2


In [7]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [8]:
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT     int64
NOM_ENT    object
CVE_MUN     int64
NOM_MUN    object
dtype: object

In [9]:
# Revisamos la cantidad de filas y columnas del dataset
dataset_inegi.shape

(299568, 4)

In [10]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
0,1,Aguascalientes,1,Aguascalientes
1,1,Aguascalientes,1,Aguascalientes
2,1,Aguascalientes,1,Aguascalientes
3,1,Aguascalientes,1,Aguascalientes
4,1,Aguascalientes,1,Aguascalientes


In [11]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
dataset_inegi.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
299563,32,Zacatecas,58,Santa María de la Paz
299564,32,Zacatecas,58,Santa María de la Paz
299565,32,Zacatecas,58,Santa María de la Paz
299566,32,Zacatecas,58,Santa María de la Paz
299567,32,Zacatecas,58,Santa María de la Paz


In [12]:
# Observamos los valores únicos y la cantidad de cada columna, ordenados
for column in dataset_inegi.columns:
    unique_values = dataset_inegi[column].unique()
    if unique_values.dtype == 'object':
        unique_values = unique_values.astype(str)
        unique_values.sort()
    if len(unique_values) <= 1000:
        print(
            f"\nCantidad de valores unicos {len(unique_values)}. Valores únicos en la columna {column}: {unique_values}")
    else:
        print(f"\nCantidad de valores únicos en {column}: {len(unique_values)}")
        print(f"Valores únicos en la columna {column}:")
        for value in unique_values:
            print(value)


Cantidad de valores unicos 32. Valores únicos en la columna CVE_ENT: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]

Cantidad de valores unicos 32. Valores únicos en la columna NOM_ENT: ['Aguascalientes' 'Baja California' 'Baja California Sur' 'Campeche'
 'Chiapas' 'Chihuahua' 'Ciudad de México' 'Coahuila de Zaragoza' 'Colima'
 'Durango' 'Guanajuato' 'Guerrero' 'Hidalgo' 'Jalisco'
 'Michoacán de Ocampo' 'Morelos' 'México' 'Nayarit' 'Nuevo León' 'Oaxaca'
 'Puebla' 'Querétaro' 'Quintana Roo' 'San Luis Potosí' 'Sinaloa' 'Sonora'
 'Tabasco' 'Tamaulipas' 'Tlaxcala' 'Veracruz de Ignacio de la Llave'
 'Yucatán' 'Zacatecas']

Cantidad de valores unicos 570. Valores únicos en la columna CVE_MUN: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62

In [13]:
dataset_inegi['CVE_ENT'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
      dtype=int64)

In [14]:
# Información general del dataset
dataset_inegi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299568 entries, 0 to 299567
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   CVE_ENT  299568 non-null  int64 
 1   NOM_ENT  299568 non-null  object
 2   CVE_MUN  299568 non-null  int64 
 3   NOM_MUN  299568 non-null  object
dtypes: int64(2), object(2)
memory usage: 9.1+ MB


In [15]:
# Filas duplicadas y filas únicas
duplicated_rows = dataset_inegi.duplicated()
number_of_duplicated_rows = duplicated_rows.sum()
print(f"El número de filas duplicadas es: {number_of_duplicated_rows}")
print(f"El número de filas únicas es: {dataset_inegi.shape[0] - number_of_duplicated_rows}")

El número de filas duplicadas es: 297092
El número de filas únicas es: 2476


In [16]:
dataset_inegi = dataset_inegi.drop_duplicates()

In [17]:
dataset_inegi['NOM_MUN_Clean'] = dataset_inegi['NOM_MUN'].apply(clean_text)
dataset_inegi['NOM_ENT_Clean'] = dataset_inegi['NOM_ENT'].apply(clean_text)

Previous text: Aguascalientes
Cleaned text: aguascalientes
Previous text: Asientos
Cleaned text: asientos
Previous text: Calvillo
Cleaned text: calvillo
Previous text: Cosío
Cleaned text: cosio
Previous text: Jesús María
Cleaned text: jesus maria
Previous text: Pabellón de Arteaga
Cleaned text: pabellon de arteaga
Previous text: Rincón de Romos
Cleaned text: rincon de romos
Previous text: San José de Gracia
Cleaned text: san jose de gracia
Previous text: Tepezalá
Cleaned text: tepezala
Previous text: El Llano
Cleaned text: el llano
Previous text: San Francisco de los Romo
Cleaned text: san francisco de los romo
Previous text: Ensenada
Cleaned text: ensenada
Previous text: Mexicali
Cleaned text: mexicali
Previous text: Tecate
Cleaned text: tecate
Previous text: Tijuana
Cleaned text: tijuana
Previous text: Playas de Rosarito
Cleaned text: playas de rosarito
Previous text: San Quintín
Cleaned text: san quintin
Previous text: San Felipe
Cleaned text: san felipe
Previous text: Comondú
Clean

In [18]:
dataset_inegi

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,NOM_MUN_Clean,NOM_ENT_Clean
0,1,Aguascalientes,1,Aguascalientes,aguascalientes,aguascalientes
708,1,Aguascalientes,2,Asientos,asientos,aguascalientes
945,1,Aguascalientes,3,Calvillo,calvillo,aguascalientes
1237,1,Aguascalientes,4,Cosío,cosio,aguascalientes
1330,1,Aguascalientes,5,Jesús María,jesus maria,aguascalientes
...,...,...,...,...,...,...
299150,32,Zacatecas,54,Villa Hidalgo,villa hidalgo,zacatecas
299211,32,Zacatecas,55,Villanueva,villanueva,zacatecas
299363,32,Zacatecas,56,Zacatecas,zacatecas,zacatecas
299484,32,Zacatecas,57,Trancoso,trancoso,zacatecas


In [19]:
dataset_inegi.isna().sum()

CVE_ENT          0
NOM_ENT          0
CVE_MUN          0
NOM_MUN          0
NOM_MUN_Clean    0
NOM_ENT_Clean    0
dtype: int64

## 2.2 Listado de Productores Autorizados

In [20]:
listado_productores.columns

Index(['ESTADO', 'MUNICIPIO', 'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
       'NOMBRE (S)', 'PAQUETE'],
      dtype='object')

In [21]:
listado_productores.head()

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000510-S000-AS,AGUILAR,AGUILAR,NICOLAS,2
1,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000339-S000-AS,AGUILAR,GARCIA,MANUEL,2
2,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000340-S000-AS,AGUILAR,GARCIA,PAULO,2
3,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000511-S000-AS,AGUILAR,OROPEZA,MIGUEL,2
4,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000676-S000-AS,ALBA,ESTRADA,MARIA DEL ROSARIO,2


In [22]:
# Mostramos el tipo de dato de cada columna
listado_productores.dtypes

ESTADO              object
MUNICIPIO           object
ACUSE               object
APELLIDO PATERNO    object
APELLIDO MATERNO    object
NOMBRE (S)          object
PAQUETE              int64
dtype: object

In [23]:
# Revisamos la cantidad de filas y columnas del dataset
listado_productores.shape

(1525720, 7)

In [24]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
listado_productores.head()

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000510-S000-AS,AGUILAR,AGUILAR,NICOLAS,2
1,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000339-S000-AS,AGUILAR,GARCIA,MANUEL,2
2,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000340-S000-AS,AGUILAR,GARCIA,PAULO,2
3,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000511-S000-AS,AGUILAR,OROPEZA,MIGUEL,2
4,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000676-S000-AS,ALBA,ESTRADA,MARIA DEL ROSARIO,2


In [25]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
listado_productores.tail()

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
1525715,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103370-S000-PL,NaN,NaN,NaN,2
1525716,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103371-S000-PL,NaN,NaN,NaN,2
1525717,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103166-S000-PL,NaN,NaN,NaN,2
1525718,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103373-S000-PL,NaN,NaN,NaN,2
1525719,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103374-S000-PL,NaN,NaN,NaN,2


In [26]:
# Observamos los valores únicos y la cantidad de cada columna, ordenados
COLUMNS_TO_VIEW = ['ESTADO', 'MUNICIPIO', 'PAQUETE']
for column in COLUMNS_TO_VIEW:
    unique_values = listado_productores[column].unique()
    if unique_values.dtype == 'object':
        unique_values = unique_values.astype(str)
        unique_values.sort()
    if len(unique_values) <= 1000:
        print(
            f"\nCantidad de valores unicos {len(unique_values)}. Valores únicos en la columna {column}: {unique_values}")
    else:
        print(f"\nCantidad de valores únicos en {column}: {len(unique_values)}")
        print(f"Valores únicos en la columna {column}:")
        for value in unique_values:
            print(value)


Cantidad de valores unicos 20. Valores únicos en la columna ESTADO: ['AGUASCALIENTES' 'CAMPECHE' 'CHIAPAS' 'CIUDAD DE MEXICO' 'COLIMA'
 'DURANGO' 'GUERRERO' 'HIDALGO' 'JALISCO' 'MEXICO' 'MICHOACAN DE OCAMPO'
 'MORELOS' 'MÉXICO' 'OAXACA' 'PUEBLA' 'QUERETARO DE ARTEAGA'
 'QUINTANA ROO' 'TLAXCALA' 'VERACRUZ DE IGNACIO DE LA LLAVE' 'YUCATAN']

Cantidad de valores únicos en MUNICIPIO: 1695
Valores únicos en la columna MUNICIPIO:
ABALA
ABEJONES
ACACOYAGUA
ACAJETE
ACALA
ACAMBAY
ACANCEH
ACAPETAHUA
ACAPULCO DE JUAREZ
ACATENO
ACATEPEC
ACATIC
ACATLAN
ACATLAN DE JUAREZ
ACATLAN DE PEREZ FIGUEROA
ACATZINGO
ACAXOCHITLAN
ACOLMAN
ACTEOPAN
ACTOPAN
ACUAMANALA DE MIGUEL HIDALGO
ACUITZIO
ACULCO
AGUA BLANCA DE ITURBIDE
AGUASCALIENTES
AGUILILLA
AHUACATLAN
AHUACUOTZINGO
AHUALULCO DE MERCADO
AHUATLAN
AHUAZOTEPEC
AHUEHUETITLA
AJACUBA
AJALPAN
AJUCHITLAN DEL PROGRESO
AKIL
ALBINO ZERTUCHE
ALCOZAUCA DE GUERRERO
ALDAMA
ALFAJAYUCAN
ALJOJUCA
ALMOLOYA
ALMOLOYA DE ALQUISIRAS
ALMOLOYA DE JUAREZ
ALMOLOYA DEL RIO
ALPOYECA

In [27]:
listado_productores

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000510-S000-AS,AGUILAR,AGUILAR,NICOLAS,2
1,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000339-S000-AS,AGUILAR,GARCIA,MANUEL,2
2,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000340-S000-AS,AGUILAR,GARCIA,PAULO,2
3,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000511-S000-AS,AGUILAR,OROPEZA,MIGUEL,2
4,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000676-S000-AS,ALBA,ESTRADA,MARIA DEL ROSARIO,2
...,...,...,...,...,...,...,...
1525715,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103370-S000-PL,NaN,NaN,NaN,2
1525716,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103371-S000-PL,NaN,NaN,NaN,2
1525717,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103166-S000-PL,NaN,NaN,NaN,2
1525718,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103373-S000-PL,NaN,NaN,NaN,2


In [28]:
# Información general del dataset
listado_productores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525720 entries, 0 to 1525719
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   ESTADO            1525720 non-null  object
 1   MUNICIPIO         1450208 non-null  object
 2   ACUSE             1525720 non-null  object
 3   APELLIDO PATERNO  1521426 non-null  object
 4   APELLIDO MATERNO  1497507 non-null  object
 5   NOMBRE (S)        1521427 non-null  object
 6   PAQUETE           1525720 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 81.5+ MB


In [29]:
# Filas duplicadas y filas únicas
duplicated_rows_productores = listado_productores.duplicated()
number_of_duplicated_rows_productores = duplicated_rows_productores.sum()
print(f"El número de filas duplicadas es: {number_of_duplicated_rows_productores}")
print(f"El número de filas únicas es: {listado_productores.shape[0] - number_of_duplicated_rows_productores}")

El número de filas duplicadas es: 0
El número de filas únicas es: 1525720


In [30]:
# Valores nulos
listado_productores.isna().sum()

ESTADO                  0
MUNICIPIO           75512
ACUSE                   0
APELLIDO PATERNO     4294
APELLIDO MATERNO    28213
NOMBRE (S)           4293
PAQUETE                 0
dtype: int64

In [31]:
# Los valores faltantes de la columna "MUNICIPIO" provienen todos del mismo estado VERACRUZ DE IGNACIO DE LA LLAVE
listado_productores[listado_productores['MUNICIPIO'].isna()]['ESTADO'].unique()
listado_productores[listado_productores['MUNICIPIO'].isna()].groupby('ESTADO').size()

ESTADO
VERACRUZ DE IGNACIO DE LA LLAVE    75512
dtype: int64

In [32]:
listado_productores[(listado_productores['MUNICIPIO'].isna())]


,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
1400822,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000001-S000-VZ,ABURTO,ALARCON,ASCENCION,1
1400823,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000002-S000-VZ,ABURTO,ALARCON,GENARO,1
1400824,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000003-S000-VZ,ABURTO,MARTINEZ,MARIA DEL CARMEN,2
1400825,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000004-S000-VZ,ABURTO,MORALES,MIGUEL ANGEL,2
1400826,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000005-S000-VZ,ALARCON,GUZMAN,MARCIANO,2
...,...,...,...,...,...,...,...
1476329,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081776-S000-VZ,REYES,HERNANDEZ,RUTILIO,1
1476330,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081842-S000-VZ,SANTIAGO,SARMIENTO,LEOBARDO,2
1476331,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081843-S000-VZ,VAZQUEZ,GAONA,JOSE,2
1476332,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081845-S000-VZ,VEGA,RODRIGUEZ,JUAN,1


In [33]:
listado_productores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525720 entries, 0 to 1525719
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   ESTADO            1525720 non-null  object
 1   MUNICIPIO         1450208 non-null  object
 2   ACUSE             1525720 non-null  object
 3   APELLIDO PATERNO  1521426 non-null  object
 4   APELLIDO MATERNO  1497507 non-null  object
 5   NOMBRE (S)        1521427 non-null  object
 6   PAQUETE           1525720 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 81.5+ MB


In [34]:
listado_productores.drop_duplicates()

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000510-S000-AS,AGUILAR,AGUILAR,NICOLAS,2
1,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000339-S000-AS,AGUILAR,GARCIA,MANUEL,2
2,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000340-S000-AS,AGUILAR,GARCIA,PAULO,2
3,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000511-S000-AS,AGUILAR,OROPEZA,MIGUEL,2
4,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000676-S000-AS,ALBA,ESTRADA,MARIA DEL ROSARIO,2
...,...,...,...,...,...,...,...
1525715,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103370-S000-PL,NaN,NaN,NaN,2
1525716,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103371-S000-PL,NaN,NaN,NaN,2
1525717,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103166-S000-PL,NaN,NaN,NaN,2
1525718,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103373-S000-PL,NaN,NaN,NaN,2


# 3. Merge de los datasets de INEGI Y PRODUCTORES AUTORIZADOS

Para el merge de los dos conjuntos de datos, primero es necesario la correción de la ortografía de los estados en el dataset de los productores autorizados, para que coincida con el dataset del INEGI.


## 3.1 Corrección de la ortografía de los estados

In [35]:
# Aseguramos el tipado antes de unir ambos datasets
# Convertimos los valores de CVE_ENT y CVE_MUN a string
DTYPE_INEGI = {
    'CVE_ENT': 'string',
    'CVE_MUN': 'string',
    'cve_ent': 'string',
    'cve_mun': 'string'
}
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype=DTYPE_INEGI)
dataset_inegi.drop(COLUMNS_TO_DROP, axis=1, inplace=True)
dataset_inegi.drop_duplicates(inplace=True)
dataset_inegi

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
0,01,Aguascalientes,001,Aguascalientes
708,01,Aguascalientes,002,Asientos
945,01,Aguascalientes,003,Calvillo
1237,01,Aguascalientes,004,Cosío
1330,01,Aguascalientes,005,Jesús María
...,...,...,...,...
299150,32,Zacatecas,054,Villa Hidalgo
299211,32,Zacatecas,055,Villanueva
299363,32,Zacatecas,056,Zacatecas
299484,32,Zacatecas,057,Trancoso


In [36]:
DTYPE_PRODUCTORES = {
    'ESTADO': 'string',
    'MUNICIPIO': 'string',
    'ACUSE': 'string',
    'APELLIDO PATERNO': 'string',
    'APELLIDO MATERNO': 'string',
    'NOMBRE (S)': 'string',
}
listado_productores.astype(DTYPE_PRODUCTORES)

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000510-S000-AS,AGUILAR,AGUILAR,NICOLAS,2
1,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000339-S000-AS,AGUILAR,GARCIA,MANUEL,2
2,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000340-S000-AS,AGUILAR,GARCIA,PAULO,2
3,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000511-S000-AS,AGUILAR,OROPEZA,MIGUEL,2
4,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000676-S000-AS,ALBA,ESTRADA,MARIA DEL ROSARIO,2
...,...,...,...,...,...,...,...
1525715,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103370-S000-PL,<NA>,<NA>,<NA>,2
1525716,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103371-S000-PL,<NA>,<NA>,<NA>,2
1525717,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103166-S000-PL,<NA>,<NA>,<NA>,2
1525718,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103373-S000-PL,<NA>,<NA>,<NA>,2


In [37]:
# Agrupamos los valores a los que se les realizará la corrección ortográfica para el posterior match
listado_productores_mun_est = pd.DataFrame({
    'ESTADO': pd.Series(listado_productores['ESTADO']),
    'MUNICIPIO': pd.Series(listado_productores['MUNICIPIO'])
})
listado_productores_mun_est.drop_duplicates(inplace=True)

inegi_mun_est = pd.DataFrame({
    'NOM_ENT': pd.Series(dataset_inegi['NOM_ENT']),
    'NOM_MUN': pd.Series(dataset_inegi['NOM_MUN'])
})
inegi_mun_est.drop_duplicates(inplace=True)

In [38]:
# Aplicamos las limpiezas
print("==== ESTADOS DE PRODUCTORES ====")
listado_productores_mun_est['ESTADO_CLEAN'] = listado_productores_mun_est['ESTADO'].apply(clean_text)
print("==== MUNICIPIOS DE PRODUCTORES ====")
listado_productores_mun_est['MUNICIPIO_CLEAN'] = listado_productores_mun_est['MUNICIPIO'].apply(clean_text)

print("==== ESTADOS DE INEGI ====")
inegi_mun_est['NOM_ENT_CLEAN'] = inegi_mun_est['NOM_ENT'].apply(clean_text)
print("==== MUNICIPIOS DE INEGI ====")
inegi_mun_est['NOM_MUN_CLEAN'] = inegi_mun_est['NOM_MUN'].apply(clean_text)

==== ESTADOS DE PRODUCTORES ====
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: AGUASCALIENTES
Cleaned text: aguascalientes
Previous text: CIUDAD DE MEXICO
Cleaned text: ciudad de mexico
Previous text: CIUDAD DE MEXICO
Cleaned text: ciudad de mexico
Previous text: CIUDAD DE MEXICO
Cleaned text: ciudad de mexico
Previous text: CIUDAD DE MEXICO
Cleaned text: ciudad de mexico
Previous text: CIUDAD DE MEXICO
Cleaned text: ciudad de mexico
Pre

In [39]:
# Creamos un dataframe intermedio para el match
# match_municipio_entidad = pd.DataFrame({
#     'prod_estado': pd.Series(listado_productores_mun_est['ESTADO']),
#     'prod_estado_clean': pd.Series(listado_productores_mun_est['ESTADO_CLEAN']),
# 
#     'prod_municipio': pd.Series(listado_productores_mun_est['MUNICIPIO']),
#     'prod_municipio_clean': pd.Series(listado_productores_mun_est['MUNICIPIO_CLEAN']),
# 
#     'inegi_nom_ent': pd.Series(inegi_mun_est['NOM_ENT']),
#     'inegi_nom_ent_clean': pd.Series(inegi_mun_est['NOM_ENT_CLEAN']),
# 
#     'inegi_nom_mun': pd.Series(inegi_mun_est['NOM_MUN']),
#     'inegi_nom_mun_clean': pd.Series(inegi_mun_est['NOM_MUN_CLEAN']),
# })
# match_municipio_entidad

In [40]:
listado_productores_mun_est.nunique()

ESTADO               20
MUNICIPIO          1694
ESTADO_CLEAN         19
MUNICIPIO_CLEAN    1694
dtype: int64

In [41]:
inegi_mun_est.nunique()

NOM_ENT            32
NOM_MUN          2332
NOM_ENT_CLEAN      32
NOM_MUN_CLEAN    2332
dtype: int64

In [42]:
# Realizamos el match

estado_clean_ratio = []
for estado in listado_productores_mun_est['ESTADO_CLEAN'].unique():

    matches = process.extract(estado, inegi_mun_est['NOM_ENT_CLEAN'].tolist(), limit=1, scorer=fuzz.ratio)
    if matches:
        print("Estado:", estado)
        print(f"Match con {matches}\n")

    print("====================")

Estado: aguascalientes
Match con [('aguascalientes', 100)]

Estado: ciudad de mexico
Match con [('ciudad de mexico', 100)]

Estado: chiapas
Match con [('chiapas', 100)]

Estado: campeche
Match con [('campeche', 100)]

Estado: colima
Match con [('colima', 100)]

Estado: durango
Match con [('durango', 100)]

Estado: michoacan de ocampo
Match con [('michoacan de ocampo', 100)]

Estado: mexico
Match con [('mexico', 100)]

Estado: morelos
Match con [('morelos', 100)]

Estado: guerrero
Match con [('guerrero', 100)]

Estado: hidalgo
Match con [('hidalgo', 100)]

Estado: jalisco
Match con [('jalisco', 100)]

Estado: oaxaca
Match con [('oaxaca', 100)]

Estado: puebla
Match con [('puebla', 100)]

Estado: quintana roo
Match con [('quintana roo', 100)]

Estado: queretaro de arteaga
Match con [('queretaro', 62)]

Estado: tlaxcala
Match con [('tlaxcala', 100)]

Estado: veracruz de ignacio de la llave
Match con [('veracruz de ignacio de la llave', 100)]

Estado: yucatan
Match con [('yucatan', 100)]



In [43]:
municipio_clean_ratio = []
for municipio, estado in zip(listado_productores_mun_est['MUNICIPIO_CLEAN'],
                             listado_productores_mun_est['ESTADO_CLEAN']):
    print(municipio)
    # matches = process.extract(municipio, inegi_mun_est['NOM_MUN_CLEAN'].tolist(), limit=1, scorer=fuzz.ratio)
    # if matches:
    #     print("Municipio:", municipio)
    #     print(f"Match con {matches}\n")
    # 
    # print("====================")

aguascalientes
asientos
calvillo
cosio
el llano
jesus maria
pabellon de arteaga
rincon de romos
san francisco de los romo
san jose de gracia
tepezala
alvaro obregon
coyoacan
cuajimalpa de morelos
iztapalapa
la magdalena contreras
milpa alta
tlahuac
tlalpan
xochimilco
acacoyagua
acala
acapetahua
aldama
altamirano
amatan
amatenango de la frontera
amatenango del valle
angel albino corzo
arriaga
bejucal de ocampo
belisario dominguez
bella vista
benemerito de las americas
berriozabal
bochil
cacahoatan
catazaja
chalchihuitan
chamula
chanal
chapultenango
chenalho
chiapa de corzo
chiapilla
chicoasen
chicomuselo
chilon
cintalapa
coapilla
comitan de dominguez
copainala
el bosque
el parral
el porvenir
emiliano zapata
escuintla
francisco leon
frontera comalapa
frontera hidalgo
honduras de la sierra
huehuetan
huitiupan
huixtan
huixtla
ixhuatan
ixtacomitan
ixtapa
ixtapangajoya
jiquipilas
jitotol
juarez
la concordia
la grandeza
la independencia
la libertad
la trinitaria
larrainzar
las margaritas
las 

In [50]:
inegi_mun_est
mathc_municipio_entidad = inegi_mun_est.merge(listado_productores_mun_est, left_on='NOM_MUN_CLEAN',
                                              right_on='MUNICIPIO_CLEAN', how='outer', indicator=True)
mathc_municipio_entidad
mathc_municipio_entidad.rename(columns={'NOM_ENT': 'ESTADO_INEGI', 'NOM_ENT_CLEAN': 'ESTADO_INEGI_CLEAN',
                                        'NOM_MUN': 'MUNICIPIO_INEGI', 'NOM_MUN_CLEAN': 'MUNICIPIO_INEGI_CLEAN',
                                        'ESTADO': 'ESTADO_PROD', 'MUNICIPIO': 'MUNICIPIO_PROD',
                                        'ESTADO_CLEAN': 'ESTADO_PROD_CLEAN', 'MUNICIPIO_CLEAN': 'MUNICIPIO_PROD_CLEAN'},
                               inplace=True)
mathc_municipio_entidad.to_excel('../../data/match_municipio_entidad.xlsx')

In [ ]:
mathc_municipio_entidad

In [ ]:
# Realizamos el match
estado_clean_ratio = []
for estado in match_municipio_entidad['estado_clean'].unique():
    if isinstance(estado, str):
        matches = process.extract(estado, match_municipio_entidad['nom_ent_clean'].tolist(), limit=1, scorer=fuzz.ratio)
        if matches:
            nom_ent = \
                match_municipio_entidad[match_municipio_entidad['nom_ent_clean'] == matches[0][0]]['nom_ent'].values[0]
        else:
            nom_ent = None
        estado_clean_ratio.append((estado, matches[0][0], matches[0][1], nom_ent))

municipio_clean_ratio = []
for _, row in match_municipio_entidad[['estado_clean', 'municipio_clean']].drop_duplicates().iterrows():
    estado, municipio = row['estado_clean'], row['municipio_clean']
    if isinstance(municipio, str):
        matches = process.extract(municipio, match_municipio_entidad['nom_mun_clean'].tolist(), limit=1,
                                  scorer=fuzz.ratio)
        if matches:
            nom_mun = \
                match_municipio_entidad[match_municipio_entidad['nom_mun_clean'] == matches[0][0]]['nom_mun'].values[0]
        else:
            nom_mun = None
        municipio_clean_ratio.append((estado, municipio, matches[0][0], matches[0][1], nom_mun))

# Convert the lists to DataFrames
ratio_estado = pd.DataFrame(estado_clean_ratio,
                            columns=['estado_clean', 'nom_ent_clean_match', 'ratio_ent', 'final_estado'])
ratio_municipio = pd.DataFrame(municipio_clean_ratio,
                               columns=['estado', 'municipio_clean', 'nom_mun_clean_match', 'ratio_mun',
                                        'final_municipio'])